In [1]:
from datetime import datetime
import pandas as pd
import sqlalchemy as sa
import sqlalchemy.orm as so
import numpy as np
import matplotlib.pyplot as plt
from hemonc_alchemy.model.hemonc_model import Hemonc_Study, Hemonc_Modality, Hemonc_Condition, Hemonc_Component_Role, Hemonc_Ref, Hemonc_Component, Hemonc_Component_Class, Hemonc_Context, Hemonc_Regimen, Hemonc_Branch_Conditional, Hemonc_Variant, Hemonc_Regimen_Part, Part_Phase, Hemonc_Cycle_Sig, Hemonc_Sig, Sig_Days, Base, component_to_class_map, variant_study_map, regimen_to_modality_map, Condition_Map
from pathlib import Path
from datetime import date
from datetime import date
from pprint import pprint
from marshmallow import Schema, fields
from typing import List


oa_configurator.config - DEBUG - Application config path set: /Users/georginakennedy/cloudstor/CBDRH/ACDN/HemOnc_Alchemy/oa_system_config.yaml
oa_configurator.config - DEBUG - Log path set: /Users/georginakennedy/cloudstor/CBDRH/ACDN/HemOnc_Alchemy/logs
oa_configurator.config - DEBUG - DB connection string: sqlite:////Users/georginakennedy/cloudstor/CBDRH/ACDN/HemOnc_Alchemy/data/dash.db


In [2]:
import omop_alchemy as oa
engine = oa.oa_config.engine

In [3]:
# probably don't do it this way - takes much longer than following option, by the time you eagerly load all the required properties for further analysis

# with so.Session(engine) as session:
#     regimens = session.query(
#         Hemonc_Regimen
#     ).filter(Hemonc_Regimen.condition_filter('C25%')).all()

In [4]:
# this is a bit slow, but it grabs literally every object eagerly for in-memory analysis - ends up being worthwhile...

with so.Session(engine) as session:
    regimens = session.query(
        Hemonc_Regimen
    ).options(
        so.joinedload(Hemonc_Regimen.variants, innerjoin=True)
          .joinedload(Hemonc_Variant.studied_in, innerjoin=True)
          .joinedload(Hemonc_Study.condition, innerjoin=True)
          .joinedload(Hemonc_Condition.condition_concept, innerjoin=True)
    ).all()


In [5]:
regimens_condition_filter = {icd10_condition:  [r for r in regimens if r.condition_filter(icd10_condition)] for icd10_condition in ['C25', 'C34']}

In [6]:
prostate_reg_filter = tuple(set([reg.regimen_cui for reg in regimens_condition_filter['C25']]))

In [7]:
with so.Session(engine) as session:
    prostate_regimens = session.query(
        Hemonc_Regimen
    ).options(
        so.joinedload(Hemonc_Regimen.component_roles, innerjoin=True)
        .joinedload(Hemonc_Component_Role.component),
        so.joinedload(Hemonc_Regimen.variants, innerjoin=True)
        .joinedload(Hemonc_Variant.has_parts, innerjoin=True)
        .joinedload(Hemonc_Regimen_Part.component_sigs, innerjoin=True)
        .joinedload(Hemonc_Sig.component, innerjoin=True)
    ).filter(Hemonc_Regimen.regimen_cui.in_(prostate_reg_filter)).all()


In [8]:
with so.Session(engine) as session:
    agents = pd.DataFrame(
        session.query(
            Hemonc_Component.component_name, 
            Hemonc_Component.component_concept_id
        ).filter(Hemonc_Component.component_concept_id.in_(prostate_regimens[0].variants[0].PO_agents)))

In [9]:
agents

,component_name,component_concept_id
0,Capecitabine,35802939
1,Erlotinib,35803050


In [10]:
# side bar to check that we can use concept table for these maps once vocab is re-imported
# looks ok mostly - very limited gaps

import dotenv, pyreadr, os
TABLE_PATH = Path('..') / 'Tables'

from omop_alchemy.model.vocabulary import Concept, Concept_Relationship

with so.Session(engine) as sess:
    cc = pd.DataFrame(
        sess.query(
            Condition_Map.condition_concept_id, 
            Condition_Map.relationship_id, 
            Condition_Map.standard_concept_code, 
            Condition_Map.standard_vocabulary_id, 
            Condition_Map.standard_concept_name
        )
    )

with so.Session(engine) as sess:
    cond = pd.DataFrame(
        sess.query(
            Concept.concept_id.label('condition_concept_id'), 
            Concept.concept_code, 
            Concept.concept_name
        ).filter(
            sa.and_(
                Concept.vocabulary_id=='HemOnc',
                Concept.concept_class_id=='Condition'
            )

        )
    )

omop = pyreadr.read_r(TABLE_PATH / 'omop.RData')
concept_relationship_columns = ['concept_code_1', 'concept_code_2', 'vocabulary_id_1', 'vocabulary_id_2', 'relationship_id', 'valid_start_date', 'valid_end_date', 'invalid_reason']
concept_relationship_stage = omop['concept_relationship_stage'][concept_relationship_columns].copy()
cond = cond.merge(cc, how='left')
with so.Session(engine) as sess:
    ho_cond = pd.DataFrame(
        sess.query(
            Hemonc_Condition.condition_concept_id,
            Hemonc_Condition.condition_code,
            Hemonc_Condition.condition_name
        )
    )

missing_maps = cond[cond.standard_vocabulary_id.isna()].merge(ho_cond, left_on='concept_code', right_on='condition_code', how='left')
new_maps = missing_maps.merge(concept_relationship_stage[['concept_code_1', 'concept_code_2', 'vocabulary_id_2', 'relationship_id']][concept_relationship_stage.relationship_id=='Maps to'], left_on='concept_code', right_on='concept_code_1', how='left')

In [11]:
new_maps[new_maps.concept_code_1.isna()]

,condition_concept_id_x,concept_code,concept_name,relationship_id_x,standard_concept_code,standard_vocabulary_id,standard_concept_name,condition_concept_id_y,condition_code,condition_name,concept_code_1,concept_code_2,vocabulary_id_2,relationship_id_y
13,912068,18799,TMB-H,NaN,NaN,NaN,NaN,NaN,18799,High tumor mutation burden malignancy,NaN,NaN,NaN,NaN
23,1525102,63998,Ewing Sarcoma pediatric,None,None,None,None,1525102.0,63998,Ewing Sarcoma pediatric,NaN,NaN,NaN,NaN
38,1525204,75336,Ph-positive B-cell acute lymphoblastic leukemi...,NaN,NaN,NaN,NaN,NaN,75336,BCR::ABL1 B-cell acute lymphoblastic leukemia ...,NaN,NaN,NaN,NaN
39,42542125,551,Acquired coagulopathy,NaN,NaN,NaN,NaN,NaN,551,Acquired hemophilia A,NaN,NaN,NaN,NaN
40,42542131,557,Allogeneic HSCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,42542140,566,Autologous HSCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,42542194,624,Mismatch repair deficient malignancy,NaN,NaN,NaN,NaN,42542194.0,624,Mismatch repair deficient malignancy,NaN,NaN,NaN,NaN
61,42542240,672,Thrombocytopenia in liver disease,NaN,NaN,NaN,NaN,42542240.0,672,Thrombocytopenia in liver disease,NaN,NaN,NaN,NaN
68,42542264,5136,BRAF-mutated malignancy,NaN,NaN,NaN,NaN,NaN,5136,Malignant solid neoplasm BRAF-mutated,NaN,NaN,NaN,NaN
69,42542274,17427,EGFR-mutated malignancy,NaN,NaN,NaN,NaN,NaN,17427,Malignant solid neoplasm EGFR-mutated,NaN,NaN,NaN,NaN


In [12]:
class Variant_Schema(Schema):
    variant_name = fields.Str()
    PO_agents = fields.List(fields.Int())
    IV_agents = fields.List(fields.Int())

class Regimen_Summary_Schema(Schema):
    regimen_name = fields.Str()
    regimen_concept_id = fields.Int()
    regimen_cui = fields.Int()
    variants = fields.List(fields.Nested(Variant_Schema))

In [13]:
class Component_Schema(Schema):
    component_concept_id = fields.Int()
    component_name = fields.String()

class Sig_Schema(Schema):
    doseMinNum = fields.Int()
    frequency = fields.Str()
    component_code = fields.Str()
    doseMaxNum = fields.Int()
    route = fields.Str()
    component = fields.Nested(Component_Schema())
    
class Regimen_Part_Schema(Schema):
    variant_cui = fields.Int()
    regimen_part_id = fields.Int()
    cycle_sig_id = fields.Str()
    timing_unit = fields.Str()
    portion = fields.Str()
    component_sigs = fields.List(fields.Nested(Sig_Schema()))

In [14]:
reg_part_dump = Regimen_Part_Schema()
regimen_summary_dump = Regimen_Summary_Schema()

In [15]:
for reg in prostate_regimens:
    pprint(regimen_summary_dump.dump(reg))

{'regimen_concept_id': 35806594,
 'regimen_cui': 33148,
 'regimen_name': 'Capecitabine and Erlotinib',
 'variants': [{'IV_agents': [],
               'PO_agents': [35803050, 35802939],
               'variant_name': 'Variant #01'}]}
{'regimen_concept_id': 35804557,
 'regimen_cui': 11673,
 'regimen_name': 'Capecitabine and Gemcitabine',
 'variants': [{'IV_agents': [35803095],
               'PO_agents': [35802939],
               'variant_name': 'Variant #01'},
              {'IV_agents': [35803095],
               'PO_agents': [35802939],
               'variant_name': 'Variant #02'},
              {'IV_agents': [35803095],
               'PO_agents': [35802939],
               'variant_name': 'Variant #03'},
              {'IV_agents': [35803095],
               'PO_agents': [35802939],
               'variant_name': 'Variant #04'},
              {'IV_agents': [35803095],
               'PO_agents': [35802939],
               'variant_name': 'Variant #05'},
              {'IV_agents':

In [16]:
# study objects provide a more nuanced link between regimen and conditions via the variants and specific condition context in which they were studied

with so.Session(engine) as session:
    reg_study_condition = pd.DataFrame(session.query(Hemonc_Regimen.regimen_cui, 
                                                     Hemonc_Regimen.regimen_name,
                                                     Hemonc_Variant.variant_name,
                                                     Hemonc_Variant.variant_cui,
                                                     Hemonc_Study.study_code,
                                                     Hemonc_Study.start,
                                                     Hemonc_Study.end,
                                                     Hemonc_Study.sponsor_type,
                                                     Hemonc_Study.enrollment_from,
                                                     Hemonc_Study.enrollment_to,
                                                     Hemonc_Ref.title,
                                                     Hemonc_Ref.pub_date,
                                                     Hemonc_Condition.condition_name
                                                     ).join(Hemonc_Variant, Hemonc_Variant.regimen_cui == Hemonc_Regimen.regimen_cui, isouter=True
                                                     ).join(variant_study_map, variant_study_map.c.variant_cui==Hemonc_Variant.variant_cui, isouter=True
                                                     ).join(Hemonc_Study, Hemonc_Study.study_code == variant_study_map.c.study_code, isouter=True
                                                     ).join(Hemonc_Ref, Hemonc_Study.study_code == Hemonc_Ref.study, isouter=True
                                                     ).join(Hemonc_Condition, Hemonc_Condition.condition_code == Hemonc_Study.condition_code, isouter=True
                                                     ).all())

In [17]:
reg_study_condition.start = pd.to_datetime(reg_study_condition.start)
reg_study_condition.end = pd.to_datetime(reg_study_condition.end)
reg_study_condition.pub_date = pd.to_datetime(reg_study_condition.pub_date)

In [18]:
# study linkages allow us to explore date of first availability for regimens - this is imperfect in the source, but we resolve this by creating a fallback option

reg_study_condition = reg_study_condition.merge(reg_study_condition.groupby('variant_cui').end.min().reset_index().rename(columns={'end': 'earliest_end'}), on='variant_cui', how='left')
reg_study_condition = reg_study_condition.merge(reg_study_condition.groupby('variant_cui').pub_date.min().reset_index().rename(columns={'pub_date': 'earliest_pub'}), on='variant_cui', how='left')
reg_study_condition = reg_study_condition.merge(reg_study_condition.groupby('variant_cui').start.min().reset_index().rename(columns={'start': 'earliest_start'}), on='variant_cui', how='left')
reg_study_condition['reference_date'] = reg_study_condition.earliest_end.combine_first(reg_study_condition.earliest_start).combine_first(reg_study_condition.earliest_pub)

In [19]:
# even with all fallbacks, some still are undated - this is correct per source
reg_study_condition[reg_study_condition.reference_date.isna()].study_code.value_counts()
reg_study_condition_with_date = reg_study_condition.dropna(subset='reference_date').copy()

In [20]:
# reg_study_condition_with_date[reg_study_condition_with_date.condition_name.str.contains('lung', case=False, na=False)].regimen_name.value_counts()

In [21]:
# the full expression of cycle and component sigs are not available in the OMOP vocabularies, and arguably cannot be made available in enough detail to compare
# precisely the prescribed baseline regimen to the drug exposure events as delivered

with so.Session(engine) as session:
    var_component_days = pd.DataFrame(session.query(Hemonc_Regimen.regimen_name,
                                                    Hemonc_Regimen.regimen_cui,  
                                                    Hemonc_Variant.variant_name, 
                                                    Hemonc_Variant.variant_cui,
                                                    Hemonc_Cycle_Sig.cycle_len_min,
                                                    Hemonc_Cycle_Sig.cycle_len_max,
                                                    Hemonc_Cycle_Sig.cycle_len_units,
                                                    Hemonc_Cycle_Sig.frequency_min,
                                                    Hemonc_Cycle_Sig.frequency_max,
                                                    Hemonc_Cycle_Sig.frequency_units,
                                                    Hemonc_Cycle_Sig.residual,
                                                    Hemonc_Regimen_Part.regimen_part_id,
                                                    Hemonc_Regimen_Part.timing,
                                                    Hemonc_Regimen_Part.timing_unit, 
                                                    Hemonc_Regimen_Part.portion,
                                                    Hemonc_Sig.frequency,
                                                    Hemonc_Sig.component_name,
                                                    Hemonc_Sig.component_role,
                                                    Hemonc_Sig.step_number,
                                                    Hemonc_Sig.route,
                                                    Hemonc_Sig.doseMinNum,
                                                    Hemonc_Sig.doseMaxNum,
                                                    Hemonc_Sig.component_class,
                                                    Hemonc_Sig.tail,
                                                    Sig_Days.sig_id,
                                                    Sig_Days.day
                                                  ).join(Hemonc_Variant, Hemonc_Variant.regimen_cui == Hemonc_Regimen.regimen_cui, isouter=True
                                                  ).join(Hemonc_Regimen_Part, Hemonc_Regimen_Part.variant_cui==Hemonc_Variant.variant_cui, isouter=True
                                                  ).join(Hemonc_Cycle_Sig, Hemonc_Cycle_Sig.cycle_sig_id==Hemonc_Regimen_Part.cycle_sig_id, isouter=True
                                                  ).join(Hemonc_Sig, sa.and_(Hemonc_Sig.variant_cui==Hemonc_Regimen_Part.variant_cui, Hemonc_Sig.regimen_part_id==Hemonc_Regimen_Part.regimen_part_id), isouter=True
                                                  ).join(Sig_Days, sa.and_(Sig_Days.variant_cui==Hemonc_Sig.variant_cui, Sig_Days.regimen_part_id==Hemonc_Sig.regimen_part_id, Sig_Days.sig_id==Hemonc_Sig.sig_id)).all())

In [181]:
unique_drugs = var_component_days.groupby('variant_cui').component_name.nunique()
drug_events = var_component_days.groupby('variant_cui').component_name.count()

In [327]:
drug_events[drug_events<200]

variant_cui
129495     60
129496     66
129497     12
129498      3
129499      7
         ... 
144942    118
144943    134
144944    134
144945      3
144946      5
Name: component_name, Length: 5953, dtype: int64

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [330]:
# df = px.data.tips()
# fig = px.histogram(unique_drugs, x="component_name")

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Histogram(x=drug_events[drug_events<200],opacity=0.5), row=1, col=1)
fig.add_trace(go.Histogram(x=unique_drugs, opacity=0.5), row=1, col=2)

fig.update_layout(
     width=800, height=400,
     title='Systemic Therapy Regimen Complexity',
     showlegend=False
)

fig.update_yaxes(title_font_family="Arial",
                 title='Number of regimens', 
                 row=1, col=1)

fig.update_xaxes(title="Prescribed event count", row=1, col=1)

fig.update_yaxes(title_font=dict(color="gray"),
                 title_font_style='italic',
                 title='', 
                 side='right',
                 row=1, col=2)

fig.update_xaxes(title="Distinct anti-cancer agents", row=1, col=2)

fig.show()

In [251]:
drug_use_dates = (
    reg_study_condition_with_date
    .merge(
        var_component_days[['regimen_cui', 'component_name']]
        .drop_duplicates()
        )
    .groupby('component_name')
    .reference_date.min()
    .reset_index()
    .sort_values('reference_date')
    .reset_index(drop=True)
)

In [332]:
date_range

69

In [334]:
fig = make_subplots(x_title='Year',specs=[[{"secondary_y": True}]])

date_range = 2020 - drug_use_dates.reference_date.dt.year.min()

fig.add_trace(go.Histogram(x=drug_use_dates[drug_use_dates.reference_date.dt.year < 2020].reference_date,
                           opacity=0.5, marker_color='#00CC96',
                           nbinsx=69, name='New systemic agents'),
                           )

fig.add_trace(go.Scatter(x=drug_use_dates[drug_use_dates.reference_date.dt.year < 2020].reference_date,
                         y=drug_use_dates[drug_use_dates.reference_date.dt.year < 2020].index,
                         line={'color':'gray', "dash": 'dot'},
                         name='Cumulative count'), secondary_y = True)

fig.update_layout(
     width=800, height=400,
     title='Introduction of novel systemic anticancer agents over time',
     showlegend=True
)

fig.update_yaxes(title_font_family="Arial",
                 title='Count', 
                 secondary_y=False)
fig.update_yaxes(title_font_family="Arial",
                 title='Cumulative', 
                 secondary_y=True)
fig.show()

In [22]:
with so.Session(engine) as session:
    reg_modalities = pd.DataFrame(session.query(Hemonc_Regimen.regimen_name,
                                                Hemonc_Regimen.regimen_cui,  
                                                Hemonc_Modality.modality_name
                                                ).join(regimen_to_modality_map, Hemonc_Regimen.regimen_cui == regimen_to_modality_map.c.regimen_cui
                                                ).join(Hemonc_Modality, Hemonc_Modality.modality_code == regimen_to_modality_map.c.modality_code).all())
                                             

In [23]:
first_reg = reg_study_condition_with_date.reference_date.min()
reg_study_condition_with_date['elapsed_years'] = reg_study_condition_with_date.reference_date.dt.year - first_reg.year
reg_study_condition_with_date['elapsed_months'] = reg_study_condition_with_date.reference_date.dt.month - first_reg.month + 12*reg_study_condition_with_date.elapsed_years


In [64]:
reg_study_modalities = reg_study_condition_with_date.merge(reg_modalities, how='left').sort_values('reference_date')[['regimen_cui', 'reference_date', 'condition_name', 'modality_name', 'elapsed_months']].drop_duplicates(subset=['regimen_cui', 'condition_name', 'modality_name'])

In [65]:
reg_study_modalities['cc'] = reg_study_modalities.sort_values(['reference_date', 'regimen_cui']).groupby('modality_name').regimen_cui.cumcount()

In [83]:
start_year = 2015
end_year = 2020

baseline_lookup = reg_study_modalities[(reg_study_modalities.reference_date.dt.year < start_year)].modality_name.value_counts().to_dict()
reg_study_modalities['reg_delta'] = reg_study_modalities.cc - reg_study_modalities.modality_name.map(baseline_lookup).fillna(0)

816

In [326]:
# visualisation for patterns of modality availability over time 


modalities_of_interest = ['Immunotherapy'] # ['Targeted therapy', 'Chemotherapy', 'Immunotherapy', 'Endocrine therapy']
regimen_modalities = reg_study_modalities[
    reg_study_modalities.modality_name.isin(modalities_of_interest) & 
    (reg_study_modalities.reference_date.dt.year >= start_year) & 
    (reg_study_modalities.reference_date.dt.year < end_year)
    ].sort_values(['reference_date', 'regimen_cui'])


df_conditions = pd.DataFrame()
for label, filt in {'Lung':'lung', 'Haematological': 'lymph|leuk|myel', 'Urothelial': 'uro|blad'}.items():
    condition_count = regimen_modalities[regimen_modalities.condition_name.str.contains(filt, case=False, na=False)].reference_date.dt.year.value_counts().reset_index()
    condition_count['condition'] = label
    df_conditions = pd.concat([df_conditions, condition_count])

other_count = regimen_modalities[~regimen_modalities.condition_name.str.contains('lymph|leuk|myel|lung|uro|blad', case=False, na=False)].reference_date.dt.year.value_counts().reset_index()
other_count['condition'] = 'Other'
df_conditions = pd.concat([df_conditions, other_count])

df_conditions['elapsed_months'] = (df_conditions.reference_date - df_conditions.reference_date.min())*12*tick_gap + regimen_modalities.elapsed_months.min() 

condition_pivot = df_conditions.pivot(index='elapsed_months', columns=['condition'])['count'].fillna(0)

# Question: instead of plotting regimens alone, should we be plotting variants? Or regimen/condition pairs?

tick_gap = 1

tick_text = list(range(regimen_modalities.reference_date.min().year, end_year, tick_gap))
tick_vals = [i*12*tick_gap + regimen_modalities.elapsed_months.min() for i in range(len(tick_text))]

fig = make_subplots(specs=[[{"secondary_y": True}]])

for condition in condition_pivot.columns:
    fig.add_trace(go.Bar(
        x=condition_pivot.index, 
        y=condition_pivot[condition], 
        name=condition,
        opacity=0.5
    ),
    secondary_y = True)


line = go.Scatter(#regimen_modalities, 
                  x=regimen_modalities[(regimen_modalities.elapsed_months < tick_vals[-1] + 6)]['elapsed_months'], 
                  y=regimen_modalities['reg_delta'], 
                  line={'color':'gray', "dash": 'dot'},
                  name='All conditions'
                  )
peak = regimen_modalities[(regimen_modalities.elapsed_months < tick_vals[-1] + 6)]['reg_delta'].max()

line2 = go.Scatter(x = [min(tick_vals)-6, max(tick_vals) + 6],
                   y = [peak]*2,
                   line={'color':'red', "width": 0.3},
                   showlegend=False,
                   fill="toself", mode='lines', opacity=0
                   )

fig.add_annotation(x=min(tick_vals), y=peak,
            text=f"{int(peak)}",
            showarrow=True,font=dict(
            family="Arial",
            size=10,
            color="red"
            ),
            opacity=0)

bar_peak = condition_pivot['Lung'].max()

fig.add_trace(line)
fig.add_trace(line2)


fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = tick_vals, 
        ticktext = tick_text,
    ),
        width=800, height=800,
        title='Availability of New Immunotherapy Regimens',
)
fig.update_xaxes(title_font_family="Arial",
                 title='Year')
fig.update_yaxes(title_font_family="Arial",
                 title='New Regimen ↔ Condition Pairs: Cumulative', 
                 secondary_y=False)
fig.update_yaxes(title_font_family="Arial",
                 title='Per year, per condition', 
                 secondary_y=True)

fig.show()

In [14]:
with so.Session(engine) as session:
    component_roles = pd.DataFrame(session.query(Hemonc_Regimen.regimen_cui,
                                                 Hemonc_Regimen.regimen_name,
                                                 Hemonc_Component_Role.relationship_id,
                                                 Hemonc_Component.component_code,
                                                 Hemonc_Component.component_name,
                                                 Hemonc_Component_Class.component_class_code,
                                                 Hemonc_Component_Class.component_class_name,
                                                 ).join(Hemonc_Component_Role, Hemonc_Regimen.regimen_cui==Hemonc_Component_Role.regimen_cui
                                                 ).join(Hemonc_Component, Hemonc_Component.component_code==Hemonc_Component_Role.component_code, isouter=True
                                                 ).join(Hemonc_Component_Class, Hemonc_Component_Class.component_class_code==Hemonc_Component_Role.component_class_code, isouter=True
                                                 ).all())
    
with so.Session(engine) as session:
    component_classes = pd.DataFrame(session.query(Hemonc_Regimen.regimen_cui,
                                                   Hemonc_Regimen.regimen_name,
                                                   Hemonc_Component_Role.relationship_id,
                                                   Hemonc_Component.component_code,
                                                   Hemonc_Component.component_name,
                                                   Hemonc_Component_Class.component_class_code,
                                                   Hemonc_Component_Class.component_class_name,
                                                   ).join(Hemonc_Component_Role, Hemonc_Regimen.regimen_cui==Hemonc_Component_Role.regimen_cui
                                                   ).join(Hemonc_Component, Hemonc_Component.component_code==Hemonc_Component_Role.component_code, isouter=True
                                                   ).join(component_to_class_map, Hemonc_Component.component_code==component_to_class_map.c.component_code, isouter=True       
                                                   ).join(Hemonc_Component_Class, Hemonc_Component_Class.component_class_code==component_to_class_map.c.component_class_code, isouter=True
                                                   ).all())

In [15]:
component_roles_with_dates = component_roles.merge(reg_study_condition_with_date[['regimen_cui', 'reference_date', 'elapsed_months']].drop_duplicates())

In [16]:
chemo_count = component_roles_with_dates[
    component_roles_with_dates.relationship_id.str.contains('chemo')
    ].groupby(
        'regimen_cui'
        ).component_name.nunique().reset_index().rename(columns={'component_name': 'chemo_n'})

chemo_count = reg_study_condition_with_date[['regimen_cui', 'regimen_name', 'reference_date', 'condition_name', 'elapsed_months']].drop_duplicates().merge(chemo_count, on='regimen_cui')
chemo_count['cc'] = chemo_count.sort_values(['reference_date', 'regimen_cui']).groupby('chemo_n').regimen_cui.cumcount()

In [ ]:
# visualisation for # of agents over time as a proxy measure for regimen complexity

tick_text = list(range(chemo_count.reference_date.min().year, date.today().year, 5))
tick_vals = [i*60 + chemo_count.elapsed_months.min() for i in range(len(tick_text))]

fig = px.line(chemo_count.sort_values(['reference_date', 'regimen_cui']),
              x='elapsed_months', 
              y='cc', 
              log_y=True, 
              color='chemo_n',
              labels={
                     'elapsed_months': 'Date',
                     'cc': 'Cumulative Regimen Count (log)',
                     'chemo_n': '# Chemo Agents'
                 },
              title='Availability of New Regimens by Number of Chemotherapy Agents')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = tick_vals, 
        ticktext = tick_text
    )
)
fig.show()

In [18]:
with so.Session(engine) as session:
    reg_study_condition = pd.DataFrame(session.query(Hemonc_Regimen.regimen_cui, 
                                                     Hemonc_Regimen.regimen_name,
                                                     Hemonc_Variant.variant_name,
                                                     Hemonc_Variant.variant_cui,
                                                     Hemonc_Study.study_code,
                                                     Hemonc_Study.start,
                                                     Hemonc_Study.end,
                                                     Hemonc_Study.sponsor_type,
                                                     Hemonc_Study.enrollment_from,
                                                     Hemonc_Study.enrollment_to,
                                                     Hemonc_Ref.title,
                                                     Hemonc_Ref.pub_date,
                                                     Hemonc_Condition.condition_code,
                                                     Hemonc_Condition.condition_name
                                                     ).join(Hemonc_Variant, Hemonc_Variant.regimen_cui == Hemonc_Regimen.regimen_cui, isouter=True
                                                     ).join(variant_study_map, variant_study_map.c.variant_cui==Hemonc_Variant.variant_cui, isouter=True
                                                     ).join(Hemonc_Study, Hemonc_Study.study_code == variant_study_map.c.study_code, isouter=True
                                                     ).join(Hemonc_Ref, Hemonc_Study.study_code == Hemonc_Ref.study, isouter=True
                                                     ).join(Hemonc_Condition, Hemonc_Condition.condition_code == Hemonc_Study.condition_code, isouter=True
                                                     ).all())